In [6]:
# Cell 1: Install Dependencies
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
!pip install pycocotools

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-zk5yo76n
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-zk5yo76n
  Resolved https://github.com/facebookresearch/detectron2.git to commit d38d7161247e64276d4c44d9d0605291e80bd969
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 63.4 MB/s eta 0:00:00
  Created wheel for detectron2: filename=detectron2-0.6-cp312-cp312-linux_x86_64.whl size=6733300 sha256=99e8179507ff52858581fcd58f028f534906b0b6058bb90518029a168897d4a4
  Stored in directory: /tmp/pip-ephem-wheel-cache-1wgadm3x/wheels/d3/6e/bd/1969578f1456a6be

In [7]:
# Cell 2: Imports
import detectron2
import torch
import os
import numpy as np

from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, HookBase
from detectron2.data import MetadataCatalog, DatasetCatalog, build_detection_test_loader
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator, SemSegEvaluator, inference_on_dataset
from pycocotools.coco import COCO
from PIL import Image
from torchvision.utils import save_image
import torchvision.transforms.functional as F

In [9]:
# Cell 3: Mount Drive and Unzip Data
from google.colab import drive
drive.mount('/content/drive')

# Ensure a clean start by removing any old data folders
!rm -rf /content/train /content/valid /content/test /content/trainmask /content/validmask

# --- IMPORTANT ---
# Change the path below to match where you saved your data.zip file in Google Drive
!unzip -q /content/drive/My\ Drive/data.zip -d /content/

print("Data unzipped successfully.")

Mounted at /content/drive
Data unzipped successfully.


In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
!unzip -q /content/drive/My\ Drive/data.zip -d /content/

print("Data unzipped successfully.")

replace /content/train/1.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace /content/train/1.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
Data unzipped successfully.


In [17]:
# Cell 4: Register COCO Datasets
register_coco_instances("train", {}, "/content/train/_annotations.coco.json", "/content/train")
register_coco_instances("valid", {}, "/content/valid/_annotations.coco.json", "/content/valid")

In [18]:
# Cell 5: Generate Segmentation Masks

# Use absolute paths for Colab
train_path = '/content/train'
train_mask_path = '/content/trainmask'
valid_path = '/content/valid'
valid_mask_path = '/content/validmask'

# --- Generate Training Masks ---
print("Generating training masks...")
os.makedirs(train_mask_path, exist_ok=True)
coco_train = COCO(os.path.join(train_path, '_annotations.coco.json'))
for img_info in coco_train.imgs.values():
    im_path = os.path.join(train_path, img_info['file_name'])
    if not os.path.exists(im_path): continue

    mask = np.zeros((img_info['height'], img_info['width']), dtype=np.uint8)
    ann_ids = coco_train.getAnnIds(imgIds=img_info['id'])
    anns = coco_train.loadAnns(ann_ids)
    for ann in anns:
        mask = np.maximum(coco_train.annToMask(ann), mask)

    mask_img = Image.fromarray(mask)
    mask_img.save(os.path.join(train_mask_path, img_info['file_name']))

# --- Generate Validation Masks ---
print("Generating validation masks...")
os.makedirs(valid_mask_path, exist_ok=True)
coco_valid = COCO(os.path.join(valid_path, '_annotations.coco.json'))
for img_info in coco_valid.imgs.values():
    im_path = os.path.join(valid_path, img_info['file_name'])
    if not os.path.exists(im_path): continue

    mask = np.zeros((img_info['height'], img_info['width']), dtype=np.uint8)
    ann_ids = coco_valid.getAnnIds(imgIds=img_info['id'])
    anns = coco_valid.loadAnns(ann_ids)
    for ann in anns:
        mask = np.maximum(coco_valid.annToMask(ann), mask)

    mask_img = Image.fromarray(mask)
    mask_img.save(os.path.join(valid_mask_path, img_info['file_name']))

print("Mask generation complete.")

Generating training masks...
loading annotations into memory...
Done (t=0.04s)
creating index...
index created!
Generating validation masks...
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Mask generation complete.


In [19]:

# Replace Cell 6 with this robust version

# --- Check and remove datasets if they already exist ---
if "trainsemseg" in DatasetCatalog.list():
  DatasetCatalog.remove("trainsemseg")
if "validsemseg" in DatasetCatalog.list():
  DatasetCatalog.remove("validsemseg")
# ----------------------------------------------------

def load_semseg_dicts(image_dir, mask_dir):
    dataset_dicts = []
    for fname in sorted(os.listdir(image_dir)):
        if not fname.endswith((".png", ".jpg")): continue
        record = {
            "file_name": os.path.join(image_dir, fname),
            "sem_seg_file_name": os.path.join(mask_dir, fname),
            "image_id": fname,
        }
        dataset_dicts.append(record)
    return dataset_dicts

# Register the training dataset
DatasetCatalog.register("trainsemseg", lambda: load_semseg_dicts("/content/train", "/content/trainmask"))
MetadataCatalog.get("trainsemseg").set(
    stuff_classes=["background", "panel"],
    evaluator_type="sem_seg",
    ignore_label=255
)

# Register the validation dataset
DatasetCatalog.register("validsemseg", lambda: load_semseg_dicts("/content/valid", "/content/validmask"))
MetadataCatalog.get("validsemseg").set(
    stuff_classes=["background", "panel"],
    evaluator_type="sem_seg",
    ignore_label=255
)

print("Datasets 'trainsemseg' and 'validsemseg' are now registered.")

Datasets 'trainsemseg' and 'validsemseg' are now registered.


In [20]:
# Cell 7: Custom Trainer, Evaluator, and Hook (Final Version)

class CocoTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "coco_eval")
        return COCOEvaluator(dataset_name, cfg, distributed=False, output_dir=output_folder)

class IOUEvaluator(SemSegEvaluator):
    def __init__(self, dataset_name, distributed=True, output_dir=None):
        super().__init__(dataset_name, distributed, output_dir)
        self.reset()

    def reset(self):
        self._scores = []

    def process(self, inputs, outputs):
        for input_data, output in zip(inputs, outputs):
            pred_mask_tensor = output['instances'].pred_masks.float().sum(dim=0) > 0
            pred_mask = pred_mask_tensor.cpu().numpy()
            gt_mask = input_data['sem_seg'].cpu().numpy()

            gt_h, gt_w = gt_mask.shape
            pred_mask_img = Image.fromarray(pred_mask.astype(np.uint8) * 255)
            pred_mask_resized = np.array(pred_mask_img.resize((gt_w, gt_h), Image.NEAREST)).astype(bool)

            intersection = np.logical_and(pred_mask_resized, gt_mask).sum()
            union = np.logical_or(pred_mask_resized, gt_mask).sum()
            iou = intersection / union if union > 0 else 1.0
            self._scores.append(iou)
    def evaluate(self):
        if len(self._scores) == 0:
            return {}
        mean_iou = np.mean(self._scores)
        return {"sem_seg": {"IoU": mean_iou}}

class IOUHook(HookBase):
    def __init__(self, trainer, val_loader, tr_loader):
        self.best_metric = -1
        self.trainer = trainer
        self._val_loader = val_loader
        self._tr_loader = tr_loader

    def after_step(self):
        next_iter = self.trainer.iter + 1
        if next_iter % self.trainer.cfg.TEST.EVAL_PERIOD == 0 or next_iter == self.trainer.max_iter:

            # --- THIS IS THE FIX ---
            # Create instances of the evaluator correctly
            val_evaluator = IOUEvaluator("validsemseg")
            train_evaluator = IOUEvaluator("trainsemseg")
            # --------------------

            val_iou_dict = inference_on_dataset(self.trainer.model, self._val_loader, val_evaluator)
            train_iou_dict = inference_on_dataset(self.trainer.model, self._tr_loader, train_evaluator)

            val_iou = val_iou_dict["sem_seg"]["IoU"]
            train_iou = train_iou_dict["sem_seg"]["IoU"]

            self.trainer.storage.put_scalar("val_iou", val_iou)
            self.trainer.storage.put_scalar("train_iou", train_iou)

            print(f"[Iter {self.trainer.iter}] VAL IOU = {val_iou:.4f}")
            print(f"[Iter {self.trainer.iter}] TRAIN IOU = {train_iou:.4f}")

            if val_iou > self.best_metric:
                self.best_metric = val_iou
                torch.save(self.trainer.model.state_dict(), os.path.join(self.trainer.cfg.OUTPUT_DIR, "best_model.pth"))

In [22]:
# Cell 8: Configure and Train
cfg = get_cfg()

# Use a standard model from the model zoo
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")

cfg.DATASETS.TRAIN = ("train",)
cfg.DATASETS.TEST = ("valid",)
cfg.DATALOADER.NUM_WORKERS = 2

cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 4000
cfg.SOLVER.STEPS = [] # No learning rate decay
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256

cfg.TEST.EVAL_PERIOD = 500
cfg.SOLVER.CHECKPOINT_PERIOD = 500

cfg.OUTPUT_DIR = "/content/output/"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# Build data loaders for the custom hook
val_loader = build_detection_test_loader(cfg, "validsemseg")
tr_loader = build_detection_test_loader(cfg, "trainsemseg")

trainer = CocoTrainer(cfg)
trainer.resume_or_load(resume=False) # Start fresh training
trainer.register_hooks([IOUHook(trainer, val_loader, tr_loader)])
trainer.train()

[08/27 20:05:57 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[08/27 20:05:57 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[08/27 20:05:57 d2.data.common]: Serializing 102 elements to byte tensors and concatenating them all ...
[08/27 20:05:57 d2.data.common]: Serialized dataset takes 0.01 MiB
[08/27 20:05:57 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[08/27 20:05:57 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[08/27 20:05:57 d2.data.common]: Serializing 771 elements to byte tensors and concatenating them all ...
[08/27 20:05:57 d2.data.common]: Serialized dataset takes 0.09 MiB


KeyboardInterrupt: 

In [27]:
# Cell for Test Data Prediction and Metrics (Corrected)
import os
from detectron2.config import get_cfg
from detectron2 import model_zoo # <-- Import model_zoo
from detectron2.data import build_detection_test_loader
from detectron2.engine import DefaultPredictor
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data.datasets import register_coco_instances

# 1. Register the test dataset
try: # Use a try-except block to prevent errors on re-running the cell
    register_coco_instances("test_dataset", {}, "/content/test/_annotations.coco.json", "/content/test")
except AssertionError:
    pass # Dataset already registered

# 2. Load the model CONFIGURATION used during training
cfg = get_cfg()
# THIS IS THE MISSING LINE: It defines the model's architecture
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))

# 3. Set custom parameters for your trained model
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.WEIGHTS = "best_model.pth" # Or the path in your Google Drive
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5

# 4. Build the test data loader
evaluator = COCOEvaluator("test_dataset", output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "test_dataset")

# 5. Run inference and evaluation
model = DefaultPredictor(cfg)
print("Starting evaluation on test data...")
results = inference_on_dataset(model.model, val_loader, evaluator)
print("Evaluation results:")
print(results)

[08/27 20:13:42 d2.data.datasets.coco]: Loaded 68 images in COCO format from /content/test/_annotations.coco.json
[08/27 20:13:42 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[08/27 20:13:42 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[08/27 20:13:42 d2.data.common]: Serializing 68 elements to byte tensors and concatenating them all ...
[08/27 20:13:42 d2.data.common]: Serialized dataset takes 0.09 MiB


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


[08/27 20:13:43 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from best_model.pth ...
Starting evaluation on test data...
[08/27 20:13:43 d2.evaluation.evaluator]: Start inference on 68 batches
[08/27 20:13:44 d2.evaluation.evaluator]: Inference done 11/68. Dataloading: 0.0011 s/iter. Inference: 0.0864 s/iter. Eval: 0.0063 s/iter. Total: 0.0939 s/iter. ETA=0:00:05
[08/27 20:13:50 d2.evaluation.evaluator]: Inference done 62/68. Dataloading: 0.0017 s/iter. Inference: 0.0883 s/iter. Eval: 0.0078 s/iter. Total: 0.0978 s/iter. ETA=0:00:00
[08/27 20:13:50 d2.evaluation.evaluator]: Total inference time: 0:00:06.332017 (0.100508 s / iter per device, on 1 devices)
[08/27 20:13:50 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:05 (0.088496 s / iter per device, on 1 devices)
[08/27 20:13:50 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[08/27 20:13:50 d2.evaluation.coco_evaluation]: Saving results to ./output/coco_instances_results